In [1]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import utils
import csv
from tqdm import tqdm
import multiprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/i574473/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
def remove_english_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    filtered_sentence = []
  
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)  
    separator = ' '    
    return separator.join(filtered_sentence)


In [3]:
tqdm.pandas(desc="progress-bar")
# Function for tokenizing
def tokenize_text(text):
    tokens = []
    text_filtered = remove_english_stopwords(text)
    for sent in nltk.sent_tokenize(text_filtered):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

train_documents = []
test_documents = []
i = 0

tags_index = {'sci-fi': 1 , 'action': 2, 'comedy': 3, 'fantasy': 4, 'animation': 5, 'romance': 6}
#Reading the file
FILEPATH = 'movies.csv'
with open(FILEPATH, 'r') as csvfile:
    moviereader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for row in moviereader:
        if i == 0:
            i += 1
            continue
        i += 1
        if i <= 2000:
            train_documents.append(TaggedDocument(words=tokenize_text(row[2]), tags=[tags_index.get(row[3], 8)] ))
        else:
            test_documents.append(TaggedDocument(words=tokenize_text(row[2]), tags=[tags_index.get(row[3], 8)]))
print(train_documents[0])

TaggedDocument<['little', 'boy', 'named', 'andy', 'loves', 'room', 'playing', 'toys', 'especially', 'doll', 'named', '``', 'woody', '``', 'but', 'toys', 'andy', 'come', 'life', 'woody', 'believes', 'life', 'toy', 'good', 'however', 'must', 'worry', 'andy', "'s", 'family', 'moving', 'woody', 'know', 'andy', "'s", 'birthday', 'party', 'woody', 'realize', 'andy', "'s", 'mother', 'gave', 'action', 'figure', 'known', 'buzz', 'lightyear', 'believe', 'toy', 'quickly', 'becomes', 'andy', "'s", 'new', 'favorite', 'toy', 'woody', 'consumed', 'jealousy', 'tries', 'get', 'rid', 'buzz', 'then', 'woody', 'buzz', 'lost', 'they', 'must', 'find', 'way', 'get', 'back', 'andy', 'moves', 'without', 'pass', 'ruthless', 'toy', 'killer', 'sid', 'phillips'], [5]>


In [4]:
cores = multiprocessing.cpu_count()

model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores, alpha=0.025, min_alpha=0.001)
model_dbow.build_vocab([x for x in tqdm(train_documents)])
train_documents  = utils.shuffle(train_documents)
model_dbow.train(train_documents,total_examples=len(train_documents), epochs=30)
def vector_for_learning(model, input_docs):
    sents = input_docs
#    targets, feature_vectors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    targets, feature_vectors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, feature_vectors
model_dbow.save('./movieModel.d2v')

100%|██████████████████████████████████| 1999/1999 [00:00<00:00, 2467455.47it/s]


In [9]:
from sklearn import preprocessing

y_train, X_train = vector_for_learning(model_dbow, train_documents)
y_test, X_test = vector_for_learning(model_dbow, test_documents)

scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# min_max_scaler = preprocessing.MinMaxScaler()
# X_train_scaled = min_max_scaler.fit_transform(X_train)
# X_test_scaled = min_max_scaler.transform(X_test)

# min_max_scaler = preprocessing.MaxAbsScaler()
# X_train_scaled = min_max_scaler.fit_transform(X_train)
# X_test_scaled = min_max_scaler.transform(X_test)

In [10]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


model = LogisticRegression(n_jobs=1, C=1e5, max_iter=2000)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

/opt/homebrew/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
print('Testing accuracy for movie plots%s' % accuracy_score(y_test, y_pred))
print('Testing F1 score for movie plots: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy for movie plots0.43207126948775054
Testing F1 score for movie plots: 0.4283289240078341
